# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Wang  ->  N. Wang  |  ['Wang']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
M. Ravet  ->  M. Ravet  |  ['M. Ravet']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
T. Henning  ->  T. Henning  |  ['T. Henning']
A. Kospal  ->  A. Kospal  |  ['A. Kospal']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
T. Kalda  ->  T. Kalda  |  ['T. Kalda']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Shi  ->  J. Shi  |  ['J. Shi']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
J. Liu  ->  J. Liu  |  ['J. Liu']
Z. Fu  ->  Z. Fu  |  ['Z. Fu']
J. Li  ->  J. Li  |  ['J. Li']
J. Liu  ->  J. Liu  |  ['J. Liu']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
J. Braspenning  ->  J. Braspenning  |  ['J. Braspenning']
Arxiv has 158 new papers today
          14 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/14 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.03228


extracting tarball to tmp_2507.03228...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.03247


extracting tarball to tmp_2507.03247... done.
Retrieving document from  https://arxiv.org/e-print/2507.03412


...

extracting tarball to tmp_2507.04834...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.05176


extracting tarball to tmp_2507.05176...

 done.


Found 175 bibliographic references in tmp_2507.05176/MPQ-gas-properties.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.03562-b31b1b.svg)](https://arxiv.org/abs/2507.03562) | **The planetary-mass-limit VLT/SINFONI library: Spectral extraction and atmospheric characterization via forward modeling**  |
|| P. Palma-Bifani, et al. -- incl., <mark>G. Chauvin</mark>, <mark>M. Ravet</mark> |
|*Appeared on*| *2025-07-08*|
|*Comments*| *Acceptance date in A&A: 25/06/2025*|
|**Abstract**|            We aim to deepen our understanding of the M-L transition on planetary-mass companions and isolated brown dwarfs, and search for evidence of possible differences between these two populations of objects. To this end, we present a set of 21 VLT/SINFONI K-band observations from five archival programs at a spectral resolution of 4000. We aim to measure atmospheric properties, such as temperature, surface gravity, and metallicity, to understand the similarities and differences between objects ranging from M5 to L5 in spectral type. We extracted the spectra of these targets with the TExTRIS code. Subsequently, we model them using ForMoSA, a Bayesian forward modeling tool for spectral analysis, exploring four families of self-consistent atmospheric models: ATMO, BT-Settl, Exo-REM, and Sonora. Here we present the spectra of our targets and the derived parameters from the atmospheric modeling. We observed a drop in effective temperature of more than 500 K as a function of spectral type at the M/L transition, likely related to limitations in the current atmospheric models. In addition, we report carbon-to-oxygen ratio measurements for three companions (2M 0103 AB b, AB Pic b, and CD-35 2722 b), which contribute to the growing list of exoplanets for which this value has been measured. In conclusion, the VLT/SINFONI Library highlights two key points. First, there is a critical need to further investigate the discrepancies among grids of spectra generated by self-consistent models, as these models yield varying results and do not uniformly explore the parameter space. Second, we do not observe apparent discrepancies in the K-band spectra between companions and isolated brown dwarfs, which potentially suggests that these super-Jupiter objects formed through a similar process; however, this warrants further investigation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.03716-b31b1b.svg)](https://arxiv.org/abs/2507.03716) | **Edge-On Disk Study (EODS) II: Thermal Structure of the Flying Saucer Disk**  |
|| S. Guilloteau, et al. -- incl., <mark>D. Semenov</mark>, <mark>T. Henning</mark>, <mark>A. Kospal</mark> |
|*Appeared on*| *2025-07-08*|
|*Comments*| *10 pages, 9 figures, to be published in Astronomy & Astrophysics*|
|**Abstract**|            Context. The dust and gas temperature in proto-planetary disks play critical roles in determining their chemical evolution and influencing planet formation processes. Aims. We attempted an accurate measurement of the dust and CO temperature profile in the edge-on disk of the Flying Saucer. Methods. We used the unique properties of the Flying Saucer, its edge-on geometry and its fortunate position in front of CO clouds with different brightness temperatures to provide independent constraints on the dust temperature. We compared it with the dust temperature derived using the radiative transfer code DiskFit and the CO gas temperature. Results. We find clear evidence for a substantial gas temperature vertical gradient, with a cold (10 K) disk mid-plane and a warmer CO layer where T(r) is 27 K at 100 au, dropping with exponent 0.3. Direct evidence for CO depletion in the mid-plane, below about 1 scale height, is also found. At this height, the gas temperature is 15-20 K, consistent with the expected CO freeze out temperature. The dust disk appears optically thin at 345 GHz, and exhibits moderate settling.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.03742-b31b1b.svg)](https://arxiv.org/abs/2507.03742) | **Deep Potential: Recovering the gravitational potential and local pattern speed in the solar neighborhood with GDR3 using normalizing flows**  |
|| <mark>T. Kalda</mark>, G. M. Green |
|*Appeared on*| *2025-07-08*|
|*Comments*| *24 pages, 13 figures*|
|**Abstract**|            The gravitational potential of the Milky Way encodes information about the distribution of all matter -- including dark matter -- throughout the Galaxy. Gaia data release 3 has revealed a complex structure that necessitates flexible models of the Galactic gravitational potential. We make use of a sample of 5.6 million upper-main-sequence stars to map the full 3D gravitational potential in a one-kiloparsec radius from the Sun using a data-driven approach called ``Deep Potential''. This method makes minimal assumptions about the dynamics of the Galaxy -- that the stars are a collisionless system that is statistically stationary in a rotating frame (with pattern speed to be determined). We model the distribution of stars in 6D phase space using a normalizing flow and the gravitational network using a neural network. We recover a local pattern speed of $\Omega_p = 28.2\pm0.1\mathrm{\,km/s/kpc}$, a local total matter density of $\rho=0.086\pm0.010\mathrm{\,M_\odot/pc^3}$ and local dark matter density of $\rho_\mathrm{DM}=0.007\pm0.011\mathrm{\,M_\odot/pc^3}$. The full 3D model exhibits spatial fluctuations, which may stem from the model architecture and non-stationarity in the Milky Way.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.03744-b31b1b.svg)](https://arxiv.org/abs/2507.03744) | **Assessing the hierarchical dynamical state of molecular gas: virial parameters from 3 to 300 pc in NGC 253**  |
|| E. K. Oakes, et al. -- incl., <mark>A. Hughes</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-07-08*|
|*Comments*| *31 pages, 16 figures, submitted to ApJ*|
|**Abstract**|            Understanding how the dynamical state of the interstellar medium (ISM) changes across spatial scales can provide important insights into how the gas is organized and ultimately collapses to form stars. To this end, we present ALMA $^{12}\mathrm{CO}(2-1)$ observations at $7$ pc ($0.4''$) spatial resolution across a $1.4~\mathrm{kpc}\times5.6~\mathrm{kpc}$ ($1'.3\times1'.3$) region located in the disk of the nearby ($D = 3.5$ Mpc), massive, star-forming galaxy NGC 253. We decompose this emission with a hierarchical, multiscale dendrogram algorithm to identify 2463 structures with deconvolved sizes ranging from $\sim3$ to $300$ pc, complete to a limiting mass of $10^4~\mathrm{M_\odot}$. By comparing the virial parameter of these structures against physical properties including size, mass, surface density, velocity dispersion, and hierarchical position, we carry out a comprehensive search for a preferred scale at which gravitationally bound structures emerge. Ultimately, we do not identify any emergent scale for bound objects in our data, nor do we find a correlation between the virial parameter and structure sizes. These findings suggest that gravitational binding cannot be used to define molecular clouds and emphasize the need for multiscale approaches to characterize the ISM.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.03412-b31b1b.svg)](https://arxiv.org/abs/2507.03412) | **SKYSURF IX -- The Cosmic Optical and Infrared Background from Integrated Galaxy Light Measurements**  |
|| S. A. Tompkins, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-07-08*|
|*Comments*| *34 pages*|
|**Abstract**|            As part of the SKYSURF Hubble Space Telescope (HST) Legacy Archival program we present galaxy number counts which yield measurements of the extragalactic background light (EBL) at 15 different wavelengths. We have processed 82,752 HST images across 23 filters into 16,686 mosaics using the same software and processing pipeline throughout. Using 17/23 filters that give reliable galaxy counts, we constrain the integrated galaxy light (IGL) with a 1.5-9\% error between 0.3 and 1.6 $\mu$m in combination with 8 bands from WAVES (Wide Area VISTA Extragalactic Survey) and DEVILS (Deep Extragalactic Visible Legacy Survey). While HST was never intended to undertake large area surveys, through extensive quality control and filtering, we were able to extract a reliable and representative sample of fields distributed across the sky. Our final catalogs cover a combined $\approx 19.6 °^2$, with individual filters covering areas ranging from $\approx 0.16-7.0 °^2$. The combination of numerous independent sight-lines and area coverage allows us to reduce cosmic variance uncertainties in deep number counts to 0.06\%-1.8\%. For the first time we are able to establish a measurement of the IGL, $\mathrm{9.07 \pm 0.35 nW m^{-2} sr^{-1}}$, at 0.59 $\mu$m using HST data. We obtain a cosmic optical background value of $ 24.45 \pm 0.50 \mathrm{nW m^{-2} sr^{-1}}$. Different techniques used to measure the COB, both directly and indirectly, have recently converged indicating that the COB arises almost exclusively from processes within galaxies. This in combination with the recent values reported from New Horizons and very high energy (VHE) constraints leaves very little room for any diffuse emission coming from outside the Milky Way.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.03740-b31b1b.svg)](https://arxiv.org/abs/2507.03740) | **J-PAS: Forecasting constraints on Neutrino Masses**  |
|| G. Rodrigues, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-08*|
|*Comments*| **|
|**Abstract**|            The large-scale structure survey J-PAS is taking data since October 2023. In this work, we present a forecast based on the Fisher matrix method to establish its sensitivity to the sum of the neutrino masses. We adapt the Fisher Galaxy Survey Code (FARO) to account for the neutrino mass under various configurations applied to galaxy clustering measurements. This approach allows us to test the sensitivity of J-PAS to the neutrino mass across different tracers, with and without non-linear corrections, and under varying sky coverage. We perform our forecast for two cosmological models: $\Lambda CDM + \sum m_\nu$ and $w_0w_a CDM + \sum m_\nu$. We combine our J-PAS forecast with Cosmic Microwave Background (CMB) data from the Planck Collaboration and Type Ia supernova (SN) data from Pantheon Plus. Our analysis shows that, for a sky coverage of 8,500 square degrees, J-PAS galaxy clustering data alone will constrain the sum of the neutrino masses to an upper limit of $\sum m_\nu < 0.32$\;eV for the $\Lambda CDM + \sum m_\nu$ model, and $\sum m_\nu < 0.36$\;eV for the $w_0w_a CDM + \sum m_\nu$ model. When combined with Planck data, the upper limit improves significantly. For J-PAS+Planck, we find $\sum m_\nu < 0.061$\;eV for the $\Lambda CDM + \sum m_\nu$ model, and for J-PAS+Planck+Pantheon Plus, we obtain $\sum m_\nu < 0.12$\;eV for the $w_0w_a CDM + \sum m_\nu$ model. These results demonstrate that J-PAS clustering measurements can play a crucial role in addressing challenges in the neutrino sector, including potential tensions between cosmological and terrestrial measurements of the neutrino mass, as well as in determining the mass ordering.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.03850-b31b1b.svg)](https://arxiv.org/abs/2507.03850) | **Ocean Tides on Asynchronously Rotating Planets Orbiting Low-mass Stars**  |
|| <mark>J. Shi</mark>, et al. |
|*Appeared on*| *2025-07-08*|
|*Comments*| **|
|**Abstract**|            Planets in the liquid-water habitable zone of low-mass stars experience large tidal forces, $10^3$ to $10^4$ times those on Earth, due to the small distance between the habitable zone and the host stars. Therefore, interior solid tides, ocean tides and atmospheric tides on these planets could be much stronger than that on Earth, but rare work has been done to explicitly simulate the ocean tides. Here, for the first time, we perform global ocean tide simulations and show that ocean tides on asynchronously rotating planets with large eccentricities can reach $\mathcal{O}(1000)\,\mathrm{m}$ in height and $\mathcal{O}(10)\,\mathrm{m\,s^{-1}}$ in flow speed. Interactions between tide and bottom topography can induce large energy dissipation, $\sim\mathcal{O}(100)\,\mathrm{W\,m^{-2}}$ in global mean. This tidal energy dissipation can strongly accelerate orbital evolution by 1-2 orders of magnitude. However, for planets with small eccentricities, the ocean tides are much weaker but still comparable to that on modern Earth. Our results suggest that ocean tides on eccentric planets orbiting low-mass stars are orders of magnitude more powerful than those on Earth and can dramatically influence surface geography and orbital evolution.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.04320-b31b1b.svg)](https://arxiv.org/abs/2507.04320) | **A gravitational acceleration model to explain the double-peaked narrow emission lines shifted in the same direction**  |
|| X. Chen, G. Liao, Q. Zheng, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-08*|
|*Comments*| **|
|**Abstract**|            In this manuscript, we propose, for the first time, an oversimplified but potentially effective gravitational acceleration model to interpret the double-peaked narrow emission lines (DPNELs) shifted in the same direction. We adopt the framework of a merging kpc-scale dual-core system in an elliptical orbit, which has an emission-line galaxy with clear narrow line regions (NLRs) merging with a companion galaxy lacking emission line features. Due to gravitational forces induced by both galaxies on the NLRs, the accelerations of the far-side and near-side NLR components may share the same vector direction when projected along the line-of-sight, leading the velocities of the observed DPNELs to shift in the same direction. Our simulations indicate that the probability of producing double-peaked features shifted in the same direction reaches 5.81% in merging kpc-scale dual core systems containing emission-line galaxies. Besides the expected results from our proposed model, we identify a unique galaxy SDSS J001050.52-103246.6, whose apparent DPNELs shifted in the same direction can be plausibly explained by the gravitational acceleration model. This proposed model provides a new path to explain DPNELs shifted in the same direction in the scenario that the two galaxies align along the line-of-sight in kpc-scale dual-core systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.04618-b31b1b.svg)](https://arxiv.org/abs/2507.04618) | **Introduction to the China Space Station Telescope (CSST)**  |
|| C. Collaboration, et al. -- incl., <mark>J. Liu</mark>, <mark>Z. Fu</mark>, <mark>J. Li</mark>, <mark>J. Liu</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-08*|
|*Comments*| *44 pages, 12 figures, 1 table*|
|**Abstract**|            The China Space Station Telescope (CSST) is a next-generation Stage-IV sky survey telescope, distinguished by its large field of view (FoV), high image quality, and multi-band observation capabilities. It can simultaneously conduct precise measurements of the Universe by performing multi-color photometric imaging and slitless spectroscopic surveys. The CSST is equipped with five scientific instruments, i.e. Multi-band Imaging and Slitless Spectroscopy Survey Camera (SC), Multi-Channel Imager (MCI), Integral Field Spectrograph (IFS), Cool Planet Imaging Coronagraph (CPI-C), and THz Spectrometer (TS). Using these instruments, the CSST is expected to make significant contributions and discoveries across various astronomical fields, including cosmology, galaxy and active galactic nuclei (AGN), the Milky Way and nearby galaxies, stars, exoplanets, Solar System objects, astrometry, and transients and variable sources. This review aims to provide a comprehensive overview of the CSST instruments, observational capabilities, data products, and scientific potential.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.03562/./figures/per_target/full_comp_CD-35_2722_b.png', 'tmp_2507.03562/./figures/per_target/radar_comp_CD-35_2722_b.png', 'tmp_2507.03562/./figures/full_teff_comparison_4grids.png', 'tmp_2507.03562/./figures/radiusselec.png']
copying  tmp_2507.03562/./figures/per_target/full_comp_CD-35_2722_b.png to _build/html/
copying  tmp_2507.03562/./figures/per_target/radar_comp_CD-35_2722_b.png to _build/html/
copying  tmp_2507.03562/./figures/full_teff_comparison_4grids.png to _build/html/
copying  tmp_2507.03562/./figures/radiusselec.png to _build/html/
exported in  _build/html/2507.03562.md
    + _build/html/tmp_2507.03562/./figures/per_target/full_comp_CD-35_2722_b.png
    + _build/html/tmp_2507.03562/./figures/per_target/radar_comp_CD-35_2722_b.png
    + _build/html/tmp_2507.03562/./figures/full_teff_comparison_4grids.png
    + _build/html/tmp_2507.03562/./figures/radiusselec.png
found figures ['tmp_2507.03716/./C-Figure-A4-channels.png', 'tmp_2507.03716/./C-Figure-

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\MSun}{\ensuremath{M_{\odot}}\xspace}$
$\newcommand{\MJup}{\ensuremath{M_{\mathrm{Jup}}}\xspace}$
$\newcommand{\RJup}{\ensuremath{R_{\mathrm{Jup}}}\xspace}$
$\newcommand{\Teff}{\ensuremath{T_{\mathrm{e\!f\!f}}}\xspace}$
$\newcommand{\logg}{\ensuremath{\log (g)}\xspace}$
$\newcommand{\met}{\ensuremath{\mathrm{[M/H]}}\xspace}$
$\newcommand{\co}{\ensuremath{\mathrm{C/O}}\xspace}$
$\newcommand{\Av}{\ensuremath{A_{\mathrm{v}}}\xspace}$
$\newcommand{\fsed}{\ensuremath{f_{\mathrm{sed}}}\xspace}$
$\newcommand{\vsini}{\hbox{v \sin (i)}\xspace}$
$\newcommand{\mic}{\ensuremath{\upmu\mathrm{m}}\xspace}$
$\newcommand{\formosa}{\texttt{ForMoSA}\xspace}$
$\newcommand{\sono}{\texttt{Sonora Diamondback}\xspace}$
$\newcommand{\atmo}{\texttt{ATMO}\xspace}$
$\newcommand{\btex}{\texttt{BT-Settl}\xspace}$
$\newcommand{\btse}{\texttt{BT-SETTL_{+ C/O}}\xspace}$
$\newcommand{\exor}{\texttt{Exo-REM}\xspace}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\theenumi}{\roman{enumi}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# The planetary-mass-limit VLT/SINFONI library$\thanks{Based on observations collected at the European Organisation for Astronomical Research in the Southern Hemisphere under ESO programs 092.C-0535(A), 092.C-0803(A), 092.C-0809(A), 093.C-0502(A), and 093.C-0829(A\&B)}$: Spectral extraction and atmospheric characterization via forward modeling

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.03562-b31b1b.svg)](https://arxiv.org/abs/2507.03562)<mark>Appeared on: 2025-07-08</mark> -  _Acceptance date in A&A: 25/06/2025_

</div>
<div id="authors">

P. Palma-Bifani, et al. -- incl., <mark>G. Chauvin</mark>, <mark>M. Ravet</mark>

</div>
<div id="abstract">

**Abstract:** Medium-resolution spectra (R $_{\lambda} \sim 1000 - 10 000$ ) at near-infrared wavelengths of young M-L objects enable the study of their atmospheric properties. Specifically, by unveiling a rich set of molecular features related to the atmospheric chemistry and physics. We aim to deepen our understanding of the M-L transition on planetary-mass companions and isolated brown dwarfs, and search for evidence of possible differences between these two populations of objects. To this end, we present a set of 21 VLT/SINFONI K-band ( $1.95 - 2.45 \mathrm{\mu}$ m) observations from five archival programs at R $_{\lambda} \sim 4000$ .We aim to measure the atmospheric properties, like the $\Teff$ , $\logg$ , $\met$ , and $\co$ , to understand the similarities and differences between objects ranging from M5 to L5 in spectral type. We extracted the spectra of these targets with the \texttt{TExTRIS} code. Subsequently, we model them using $\formosa$ , a Bayesian forward modeling tool for spectral analysis, exploring four families of self-consistent atmospheric models: $\atmo$ , $\btex$ , $\exor$ , and $\sono$ . Here we present the spectra of our targets and the derived parameters from the atmospheric modeling. We confirm a drop in $\Teff$ as a function of the spectral type of more than 500 K at the M/L transition. In addition, we report $\co$ measurements for 3 companions (2M 0103 AB b, AB Pic b, and CD-35 2722 b), which add to the growing list of exoplanets with measured $\co$ ratios. The VLT/SINFONI Library highlights two key points.First, there is a critical need to further investigate the discrepancies among grids of spectra generated by self-consistent models, as these models yield varying results and do not uniformly explore the parameter space.Second, we do not observe obvious discrepancies in the K-band spectra between companions and isolated brown dwarfs, which potentially suggests that these super-Jupiter objects formed through a similar process; however, this warrants further investigation.

</div>

<div id="div_fig1">

<img src="tmp_2507.03562/./figures/per_target/full_comp_CD-35_2722_b.png" alt="Fig9.1" width="50%"/><img src="tmp_2507.03562/./figures/per_target/radar_comp_CD-35_2722_b.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** Best-fit results from each atmospheric grid using the setup of run v07 for the companion CD-35 2722 b.
On the left, we display the observed spectrum (black) and the best-fit models from each grid in the different colors, offset vertically for clarity. The $\chi^2_{red}$ values for each fit are shown on the right, on top of each model fit. The bottom-left panel shows the residuals of each fit.
On the right, a spider plot compares the derived posterior values among the grids, with the light-colored area representing 99.73\% confidence intervals on the posteriors.
For this target, all grids except ATMO perform similarly in terms of $\chi^2_{red}$ and best posterior values. The $\Teff$ values are consistent at approximately 1650 K, while $\logg$ varies between 4.0 and 4.3 dex. (*fig:target_CD-35_2722_b*)

</div>
<div id="div_fig2">

<img src="tmp_2507.03562/./figures/full_teff_comparison_4grids.png" alt="Fig10" width="100%"/>

**Figure 10. -** Best spectral fit for each target. Here, the targets are organized by their derived $\Teff$, from hottest at the top to coldest at the bottom. The $\Teff$ and S/N values are shown on the right side of each spectrum. The models are color-coded to indicate the grid that provided the best match to the data, selected based on the lowest $\chi^2_{red}$ value. (*fig:full_res_teff*)

</div>
<div id="div_fig3">

<img src="tmp_2507.03562/./figures/radiusselec.png" alt="Fig7" width="100%"/>

**Figure 7. -** Example of optimal aperture radius selection and spectra rejection criteria using the AB Pic b dataset. In each panel, each color represents a different exposure of AB Pic b.
**Left:** Here, we show the values for the fitted continuum slope between 2.05 and 2.2 $\mic$ as a function of aperture radius. Two distinct domains are observed: for radii smaller than the optimal value, the slope increases steeply; for radii larger than the optimal value, the slope still varies but becomes more stable.
**Center:** Here, we show the signal-to-noise ratio (S/N) estimated within the same wavelength range as in the left panel, plotted as a function of aperture radius. For each observation, we highlighted the maximum S/N value with a square. The aperture radius corresponding to this maximum is our final selected radius each time.
**Right:** Here, we show the derivative of the slope (reported in the left panel) with respect to the aperture radius. Some targets exhibit a non-smooth, decreasing derivative; we reject those epochs. For AB Pic b, this resulted in the rejection of the entire dataset for the second night.
 (*fig:red_radselec*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.03562"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vlsr}{v_{\rm lsr}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\mdot}{\dot{M}}$
$\newcommand{\Msun}{ \mathrm{M}_\odot}$
$\newcommand{\Msunpyr}{\unit{M_\odot yr^{-1}}}$
$\newcommand{\coto}{\mbox{CO(2--1)}}$
$\newcommand{\cott}{\mbox{CO(3--2)}}$
$\newcommand{\tcott}{\mbox{^{13}CO(2--1)}}$
$\newcommand{\csff}{\mbox{CS(5--4)}}$
$\newcommand{\csss}{\mbox{CS(7--6)}}$
$\newcommand{\cnto}{\mbox{CN(2--1)}}$
$\newcommand{\cntt}{\mbox{CN(3--2)}}$
$\newcommand{\oldcaption}[1]$</div>



<div id="title">

# Edge-On Disk Study (EODS) II: Thermal Structure of the Flying Saucer Disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.03716-b31b1b.svg)](https://arxiv.org/abs/2507.03716)<mark>Appeared on: 2025-07-08</mark> -  _10 pages, 9 figures, to be published in Astronomy & Astrophysics_

</div>
<div id="authors">

S. Guilloteau, et al. -- incl., <mark>D. Semenov</mark>, <mark>T. Henning</mark>, <mark>A. Kospal</mark>

</div>
<div id="abstract">

**Abstract:** The dust and gas temperature in proto-planetary disks play critical roles in determining   their chemical evolution and influencing planet formation processes. We attempted an accurate measurement of the dust and CO temperature profile in the edge-on disk of the Flying Saucer. We used the unique properties of the Flying Saucer, its edge-on geometry and its fortunate position in front of CO clouds with different brightness temperatures to provide independent constraints on the dust temperature. We compared it with the dust temperature derived using the radiative transfer code DiskFit and the CO gas temperature. We find clear evidence for a substantial gas temperature vertical gradient, with a cold (10 K) disk mid-plane and a warmer CO layer where $T(r) \approx 27 (r/100 \mathrm{au})^{-0.3}$ K.   Direct evidence for CO depletion in the mid-plane, below about 1 scale height, is also found.   At this height, the gas temperature is 15-20 K, consistent with the expected CO freeze out temperature.   The dust disk appears optically thin at 345 GHz, and exhibits moderate settling.

</div>

<div id="div_fig1">

<img src="tmp_2507.03716/./C-Figure-A4-channels.png" alt="Fig8" width="100%"/>

**Figure 8. -** Channel maps of CO emission.
Contour levels are in steps of 4.1 K ($\simeq 5\sigma$).
Negative signal in blue, positive in red, with color scale
spanning -23 to 23 K. Panels are labeled by the velocity, in range -0.95 to 8.42
km s$^{-1}$.
 (*fig:channels*)

</div>
<div id="div_fig2">

<img src="tmp_2507.03716/./C-Figure-2-profile.png" alt="Fig2" width="100%"/>

**Figure 2. -** Dust and CO temperature derivation. Numbers indicate the intrinsic temperatures
corresponding to the CO cloud brightness temperature at the positions of nulls
in the East (blue) and West (red) average mid-plane spectra. Black values are those
obtained for the dust. (*fig:midplane*)

</div>
<div id="div_fig3">

<img src="tmp_2507.03716/./C-Figure-A2-radial-cont.png" alt="Fig4" width="100%"/>

**Figure 4. -** Radial distribution of the brightness at 235  and 345 GHz along the disk mid-plane (continuous black and blue curves). Dashed lines indicate the best fit model prediction. The brightness ratio is given by the red curve, with $\pm 1 \sigma$ confidence interval in pink (*fig:radial*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.03716"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Deep Potential: Recovering the gravitational potential and local pattern speed in the solar neighborhood with GDR3 using normalizing flows

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.03742-b31b1b.svg)](https://arxiv.org/abs/2507.03742)<mark>Appeared on: 2025-07-08</mark> -  _24 pages, 13 figures_

</div>
<div id="authors">

<mark>T. Kalda</mark>, G. M. Green

</div>
<div id="abstract">

**Abstract:** The gravitational potential of the Milky Way encodes information about the distribution of all matter -- including dark matter -- throughout the Galaxy. _Gaia_ data release 3 has revealed a complex structure that necessitates flexible models of the Galactic gravitational potential.We make use of a sample of 5.6 million upper-main-sequence stars to map the full 3D gravitational potential in a one-kiloparsec radius from the Sun using a data-driven approach called "Deep Potential". This method makes minimal assumptions about the dynamics of the Galaxy -- that the stars are a collisionless system that is statistically stationary in a rotating frame (with pattern speed to be determined). We model the distribution of stars in 6D phase space using a normalizing flow and the gravitational network using a neural network. We recover a local pattern speed of $\Omega_p = 28.2\pm0.1\mathrm{ km/s/kpc}$ , a local total matter density of $\rho=0.086\pm0.010\mathrm{ M_\odot/pc^3}$ and local dark matter density of $\rho_\mathrm{DM}=0.007\pm0.011\mathrm{ M_\odot/pc^3}$ . The full 3D model exhibits spatial fluctuations, which may stem from the model architecture and non-stationarity in the Milky Way.

</div>

<div id="div_fig1">

<img src="tmp_2507.03742/./dpot_diagram.png" alt="Fig5" width="100%"/>

**Figure 5. -** Illustration of the various models trained for Deep Potential, which are used to both determine a best estimate of the potential and to quantify its uncertainties. Squares indicate models, circles indicate datasets, nodes colored orange pertain to the distribution function, and nodes colored blue pertain to the gravitational potential and pattern speed $\Omega_p$.
    To obtain the best-fit value for the gravitational potential and $\Omega_p$, we first train 16 normalizing flows (with different random seeds) to approximate the distribution function. We obtain a best estimate of the distribution function and its gradients by taking the median of the 16 flows and of their gradients (Ensemble $\{\pdv*{f}{\vec x}, \pdv*{f}{\vec v}, f\})$. We use this best-fit distribution function to train 16 neural networks and pattern speeds (with different random seeds). The best-fit pattern speed and potential (or its derivatives) are defined by the median of these 16 pattern speeds and neural networks (or of their derivatives).
    We additionally estimate various sources of errors in the potential. For example, to quantify the effect of observational noise, we train 8 distribution functions on Gaia data with added noise. For each distribution function, we train a potential. From this ensemble, we can calculate variance in the inferred pattern speed, potential and derived quantities.
    From top to bottom on the figure, the following uncertainties are quantified: observational errors in $\vec x_0$ and $\vec v_0$ by resampling the respective quantities based on variations in the literature values, fluctuations in the random seed and convergence of the normalizing flow and neural network architecture, observational errors in the source data by resampling the 6D positions of stars based on their quoted errors, and finally, statistical uncertainties from the finite size of the dataset using bootstrap on the training data. (*fig:dpot_diagram*)

</div>
<div id="div_fig2">

<img src="tmp_2507.03742/./scalar_values.png" alt="Fig6" width="100%"/>

**Figure 6. -** Distribution of scalar value quantities from different uncertainty estimation pipelines visualized in Figure \ref{fig:dpot_diagram}. The scalar quantities are: the average acceleration in the solar neighborhood $a_{\odot,x}, a_{\odot,y}, a_{\odot,z}$, the pattern speed of the solar neighborhood within \SI{1}{kpc} from the Sun, and the average total matter density $\rho(R=R_0)$ at the solar distance $R = R_0$ from the Galactic Center. The bottom right diagram visualizes which volumes the quantities are averaged over. The central \SI{0.1}{kpc} and outer \SI{0.8}{kpc} are excluded due to a lack of data and systematics from edge effects, respectively. In each subplot, the columns represent the spread of values due to the following five effects: adding observational noise to the observed 6D kinematics of the training data ("added observational noise"), resampling the stars in the training data using bootstrap ("bootstrap"), variations in the starting seed of the normalizing flows representing the DF ("DF seed"), variations in the starting seed of the neural network representing the gravitational potential while using the best-fit DF ("potential seed"), and adding observational noise to the literature values of $\vec x_0, \vec v_0$ while using the best-fit DF ("$\vec x_0, \vec v_0$ noise"). The best-fit value is calculated using the median of "potential seed". For discussion, see \ref{appendix:uncertainties} (*fig:scalar_values*)

</div>
<div id="div_fig3">

<img src="tmp_2507.03742/./DF_marginal_combined.png" alt="Fig9" width="100%"/>

**Figure 9. -** A demonstration of the performance of our normalizing flow model of the stellar phase-space distribution function in three different projections. We plot 2D histograms of the selected stars (left column) and of $2^{21}=2097152$ points sampled from the trained normalizing flow (middle column), as well as a comparison between the two (right column). The top row shows a face-on view of the \SI{1000}{pc} dataset, projected onto the $x-y$ plane. The middle row shows one velocity-space projection ($v_{\phi}$ vs. $v_R$), and the bottom row shows the sky projection (in a Lambert cylindrical equal-area projection: $\ell$ vs. $\sin b$). The density of the sampled normalizing flow points has been renormalized by an overall constant to match the density of the stars in the training set. For each bin, we define the Poisson significance as $(n_\mathrm{NF} - n_\mathrm{data})/\sqrt{n_\mathrm{data}}$, where $n_\mathrm{NF}$ is the renormalized number of samples in the bin drawn from the normalizing flow and $n_\mathrm{data}$ is the number of sources in the dataset in the same bin. As can be seen above, our normalizing flow captures nearly all prominent features in the stellar distribution, with the exception of some residuals seen in areas of high dust extinction in the sky projection. We thus obtain a smooth, differentiable representation of the Galactic stellar population. (*fig:df*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.03742"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\co}{^{12}\mathrm{CO}(2-1)}$
$\newcommand{\avir}{\alpha_\mathrm{vir}}$
$\newcommand{\rvir}{R_\mathrm{vir}}$
$\newcommand{\onesigma}{16--84\%}$
$\newcommand{\mco}{M_\mathrm{CO}}$
$\newcommand{\sigmol}{\Sigma_\mathrm{mol}}$
$\newcommand{\sigv}{\sigma_\mathrm{v}}$
$\newcommand{\msun}{\mathrm{M_\odot}}$
$\newcommand{\alphaco}{\alpha_\mathrm{CO}}$
$\newcommand{\rtwoone}{R_{21}}$
$\newcommand{\UConn}{\affiliation{Department of Physics, University of Connecticut, 196A Auditorium Road, Storrs, CT 06269, USA}}$
$\newcommand{\Alberta}{\affiliation{Department of Physics, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}}$
$\newcommand{\OSU}{\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, OH 43210, USA}}$
$\newcommand{ÇAPP}{\affiliation{Center for Cosmology and Astroparticle Physics (CCAPP), 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{\MPIA}{\affiliation{Max Planck Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\Princeton}{\affiliation{Department of Astrophysical Sciences, Princeton University, 4 Ivy Lane, Princeton, NJ 08544, USA}}$
$\newcommand{\JHU}{\affiliation{Department of Physics and Astronomy, Johns Hopkins University, Baltimore, MD 21218, USA}}$
$\newcommand{\UBonn}{\affiliation{Argelander-Institut für Astronomie, University of Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}}$
$\newcommand{\CfA}{\affiliation{Center for Astrophysics \mid Harvard \& Smithsonian, 60 Garden St., Cambridge, MA 02138, USA}}$
$\newcommand{\ESO}{\affiliation{European Southern Observatory, Karl-Schwarzschild-Strasse 2, 85748 Garching bei München, Germany}}$
$\newcommand{\HD}{\affiliation{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, 69120 Heidelberg, Germany}}$
$\newcommand{\UOA}{\affiliation{Department of Physics, University of Arkansas, 226 Physics Building, 825 West Dickson Street, Fayetteville, AR 72701, USA}}$
$\newcommand{\UCT}{\affiliation{Department of Astronomy, University of Cape Town, Rondebosch 7701, South Africa}}$
$\newcommand{\ESOcl}{\affiliation{European Southern Observatory (ESO), Alonso de Córdova 3107, Casilla 19, Santiago 19001, Chile}}$
$\newcommand{\OAN}{\affiliation{Observatorio Astron{ó}mico Nacional (IGN), C/Alfonso XII 3, Madrid E-28014, Spain}}$
$\newcommand{\ITA}{\affiliation{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{\UWyoming}{\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\UGent}{\affil{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{\Oxford}{\affil{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}}$
$\newcommand{\Carnegie}{\affiliation{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{\UChile}{\affiliation{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{\IRAP}{\affiliation{IRAP/OMP/Université de Toulouse, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\LUX}{\affiliation{LUX, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universités, 75014 Paris, France}}$</div>



<div id="title">

# Assessing the hierarchical dynamical state of molecular gas: virial parameters from 3 to 300 pc in NGC 253

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.03744-b31b1b.svg)](https://arxiv.org/abs/2507.03744)<mark>Appeared on: 2025-07-08</mark> -  _31 pages, 16 figures, submitted to ApJ_

</div>
<div id="authors">

Elias~K.~Oakes, et al. -- incl., <mark>A. Hughes</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Understanding how the dynamical state of the interstellar medium (ISM) changes across spatial scales can provide important insights into how the gas is organized and ultimately collapses to form stars. To this end, we present ALMA $\co$ observations at $7$ pc ( $0.4\arcsec$ ) spatial resolution across a $1.4 \mathrm{kpc}\times5.6 \mathrm{kpc}$ ( $1'.3\times1'.3$ ) region located in the disk of the nearby ( $D = 3.5$ Mpc), massive, star-forming galaxy NGC 253. We decompose this emission with a hierarchical, multiscale dendrogram algorithm to identify 2463 structures with deconvolved sizes ranging from $\sim3$ to $300$ pc, complete to a limiting mass of $10^4$  $\msun$ . By comparing the virial parameter of these structures against physical properties including size, mass, surface density, velocity dispersion, and hierarchical position, we carry out a comprehensive search for a preferred scale at which gravitationally bound structures emerge. Ultimately, we do not identify any emergent scale for bound objects in our data, nor do we find a correlation between the virial parameter and structure sizes. These findings suggest that gravitational binding cannot be used to define molecular clouds and emphasize the need for multiscale approaches to characterize the ISM.

</div>

<div id="div_fig1">

<img src="tmp_2507.03744/figures/fig_virial_radius_norvir.png" alt="Fig8" width="100%"/>

**Figure 8. -** **Left:** The virial parameter against structure radius for all leaves (blue) and branches (orange) in the dendrogram. Each point's lightness is scaled by a Gaussian kernel density estimator (KDE) to reflect the smoothed density distribution. Bins of 0.15 dex are shown with their medians in black and $\onesigma$ ranges in gray. In red, we fit the centers of these bins and show each iteration of a Bayesian linear regression using \texttt{linmix}, finding a slope of $-0.10\pm0.08$. We exclude the largest-radius bin because it is strongly affected by a single, very large lineage with duplicate flux, discussed more in \S \ref{subsec:disc_largescale}. The two dashed, horizontal lines in gray show $\alpha_\mathrm{vir} = 1$ and $2$. The estimated 50\% completeness limit for structures of typical $\sigmol$ and $\avir$(\S \ref{subsec:completeness}) is shown by the hatched boundary. $\avir$ is remarkably flat with $R$, for structures from $\sim3$ to 250 pc. **Right:** Virial parameter distributions for leaves (blue) and branches (orange). Median values are indicated by the horizontal, colored lines. (*fig:virial_radius*)

</div>
<div id="div_fig2">

<img src="tmp_2507.03744/figures/fig_levelprops_dendrogram_cartoon.png" alt="Fig7" width="100%"/>

**Figure 7. -** **Main panel:** Full dendrogram of the NGC 253 field, showing 2463 structures ordered by peak intensity within a lineage. The virial parameter $\avir$ is plotted within each structure as a function of the contour intensity level, showing a substantial decrease with intensity both between (\S \ref{subsubsec:virial_hierarchical}) and within (\S \ref{subsubsec:avir_intensity_levelprops}) individual structures. The trunks corresponding to base structures \#1018 (orange), 1560 (pink), 1439 (yellow), 488 (green), and 87 (cyan) are indicated at the bottom, using the same colors as Fig. \ref{fig:zoomfield}, panel (b).
    **Inset panels:** A subset of the dendrogram (left) and corresponding contours over the peak intensity map (right) highlight the tree-like hierarchy of the dendrogram, in which leaves (the high-intensity, peak-level structures) are nested within lower-intensity parent branches, which can in turn be nested within their own parent structures. In these panels, the orange gradient represents each structure's level. (*fig:levelprops_dendrogram*)

</div>
<div id="div_fig3">

<img src="tmp_2507.03744/figures/fig_physical_params_norvir.png" alt="Fig9" width="100%"/>

**Figure 9. -** Similar to Fig. \ref{fig:virial_radius} but with $\avir$ plotted against structure mass ($\mathrm{M_{CO}}$), surface density ($\sigmol$), and velocity dispersion ($\sigv$) for 12 bins. \texttt{linmix} fits above the completeness limits return $\alpha_\mathrm{vir} \propto M_\mathrm{CO}^{-0.11\pm0.03}$ below $\log_\mathrm{10}\left(\mathrm{M_{CO}}\right) \lesssim 7.3$ and $\alpha_\mathrm{vir} \propto \Sigma_\mathrm{mol}^{-0.58\pm0.16}$. (*fig:physical_params*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.03744"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\oiii}[1]{\text{[O III]\lambda5007}\if\relax\detokenize{#1}\relax\else\xspace\fi#1}$
$\newcommand{\nii}[1]{\text{[N II]\lambda6583}\if\relax\detokenize{#1}\relax\else\xspace\fi#1}$
$\newcommand{\ha}[1]{\text{H\alpha\if\relax\detokenize{#1}\relax\else\xspace\fi#1}}$
$\newcommand{\hb}[1]{\text{H\beta\if\relax\detokenize{#1}\relax\else\xspace\fi#1}}$
$\newcommand{\thefigure}{A1\alph{figure}}$
$\newcommand{\thefigure}{A2\alph{figure}}$
$\newcommand{\thefigure}{A3\alph{figure}}$
$\newcommand{\thefigure}{A4\alph{figure}}$
$\newcommand{\thefigure}{A5\alph{figure}}$
$\newcommand{\thefigure}{A5\alph{figure}}$
$\newcommand{\thefigure}{A6\alph{figure}}$
$\newcommand{\thefigure}{A7\alph{figure}}$
$\newcommand{\thefigure}{A8\alph{figure}}$
$\newcommand{\thefigure}{A9\alph{figure}}$</div>



<div id="title">

# Extreme cloud collisions in nearby barred galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.04530-b31b1b.svg)](https://arxiv.org/abs/2507.04530)<mark>Appeared on: 2025-07-08</mark> -  _12 pages, 7 figures, accepted for publication in MNRAS_

</div>
<div id="authors">

T. Kolcu, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** The inner regions of the Milky Way are known to contain an enigmatic population of prominent molecular clouds characterised by extremely broad lines. The physical origin of these "extended velocity features" (EVFs) is still debated, although a connection with the "dust lanes" of the Galactic bar has been hypothesised. In this paper, we search for analogous features in the dust lanes of nearby barred galaxies using the PHANGS-ALMA CO(2-1) survey. We aim to confirm existence of EVFs in other galaxies and to take advantage of the external perspective to gain insight into their origin. We study a sample of 29 barred galaxies and find that 34 \% contain one or more EVFs, while the remaining lack obvious signs of EVFs. Upon analysing the physical properties of the EVFs, we find they possess large virial parameters, ranging from few hundreds to several thousand, indicating that they are strongly out-of-equilibrium. The most likely explanation for their origin is extreme cloud-cloud collisions with relative velocities in excess of 100km/s in highly non-circular flow driven by the bar. This interpretation is consistent with previous high-resolution observations in Milky Way. Further corroboration of this interpretation comes from the inspection of high-sensitivity infrared observations from the PHANGS-JWST Treasury Survey that reveals streams of gas that appear to be hitting the dust lanes at locations where EVFs are found. We argue that EVFs are the clearest examples of cloud-cloud collisions available in literature and represent a unique opportunity to study cloud collisions and their impact on star formation.

</div>

<div id="div_fig1">

<img src="tmp_2507.04530/./Figures/EVF_nativeres/NGC1300_maps.png" alt="Fig2" width="100%"/>

**Figure 2. -** The diagnostic maps and 7.7$\mu$m emission image of NGC 1300. _Top:_ Position-position (PP) and Position-velocity (PV) diagrams. The galaxy is rotated in the plane of the sky such that the PA of the LON aligns with the _x_-axis. Red and blue colours highlight the dust lanes.
    _Bottom:_ 7.7$\mu$m emission in gray-scale, with CO(2--1) intensity as in PP diagram in yellow contours. Magenta lines mark the streams that appear to extend over large radii and reach to the regions where EVFs are seen. In all panels, the rectangular boxes mark the regions of EVFs. (*fig:NGC1300*)

</div>
<div id="div_fig2">

<img src="tmp_2507.04530/./Figures/MW/MW_subplots_contourlevels.png" alt="Fig1" width="100%"/>

**Figure 1. -** CO(1-0) emission maps of the central regions of the Milky Way. The top and bottom panels display the longitude-latitude $(l,b)$ and longitude-velocity $(l,v_{\rm los})$ CO(1-0) emission maps respectively from the survey by [Bitran, et. al (1997)](https://ui.adsabs.harvard.edu/abs/1997A&AS..125...99B). The two most prominent EVFs of the Galaxy, located at $l=5.4^\circ$ and $l=3.2^\circ$, are marked with magenta boxes and are coloured in indigo and green respectively. Red and blue are the emission from the bar dust lanes on the redshifted (DL1) and blueshifted (DL2) sides of the Galaxy. The orange box marks the Central Molecular Zone (CMZ). The Galactic centre is located at $(l,b) = (0,0)$. (*fig:MilkyWay*)

</div>
<div id="div_fig3">

<img src="tmp_2507.04530/./Figures/EVF_zoomin/NGC1300_zoom-min.png" alt="Fig3" width="100%"/>

**Figure 3. -** Close-up views of the EVF regions, marked with rectangular boxes in the bottom panel of Fig. \ref{fig:NGC1300}. Both panels show the 7.7 $\rm \mu m$ emission in gray-scale overlaid with CO(2–1) emission in yellow contours. (*fig:JWST-Zoom-NGC1300*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.04530"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kB}{\ensuremath{  {k_{\rm B}}}}$
$\newcommand{\Rtwohc}{R_{\rm 200c}}$
$\newcommand{\Rhalo}{R_{\rm halo}}$
$\newcommand{\Mtwohc}{M_{\rm 200c}}$
$\newcommand{\Rtwohm}{R_{\rm 200m}}$
$\newcommand{\Mtwohm}{M_{\rm 200m}}$
$\newcommand{\Rfivehc}{R_{\rm 500c}}$
$\newcommand{\Mfivehc}{M_{\rm 500c}}$
$\newcommand{\Mfivehcspec}{M_{\rm 500c,spec}}$
$\newcommand{\Mvir}{M_{\rm vir}}$
$\newcommand{\eg}{{\sl e.g.},}$
$\newcommand{\Rtwoh}{R_{\rm 200c}}$
$\newcommand{\rtwoh}{R_{\rm 200c}}$
$\newcommand{\mtwoh}{M_{\rm 200c}}$
$\newcommand{\Mtwoh}{M_{\rm 200c}}$
$\newcommand{\mfiveh}{M_{\rm 500c}}$
$\newcommand{\LX}{L_{\rm X}}$
$\newcommand{\LXce}{L_{\rm X, ce}}$
$\newcommand{\LXbolce}{L_{\rm X, ce}^{\rm bol}}$
$\newcommand{\LXsoft}{L_{\rm X, soft}}$
$\newcommand{\Mgas}{M_{\rm gas}}$
$\newcommand{\fgas}{f_{\rm gas}}$
$\newcommand{\fb}{f_{\rm b}}$
$\newcommand{\Tsl}{T_{\rm sl}}$
$\newcommand{\Tmw}{T_{\rm mw}}$
$\newcommand{\Tew}{T_{\rm ew}}$
$\newcommand{\Tspec}{T_{\rm spec}}$
$\newcommand{\YX}{Y_{\rm X}}$
$\newcommand{\TX}{T_{\rm X}}$
$\newcommand{\TXce}{T_{\rm X, ce}}$
$\newcommand{\YSZ}{Y_{\rm SZ}}$
$\newcommand{\Yint}{Y^{\rm int}}$
$\newcommand{\mgas}{M_{\rm gas}}$
$\newcommand{\Mstargal}{M_{\star, \rm sat}}$
$\newcommand{\mstargal}{M_{\star, \rm sat}}$
$\newcommand{\MstarBCG}{M_{\star,\rm BCG}}$
$\newcommand{\mstarBCG}{M_{\star,\rm BCG}}$
$\newcommand{\Mr}{M_{\rm r}}$
$\newcommand{\mhalo}{M_{\rm halo}}$
$\newcommand{\Mhalo}{M_{\rm halo}}$
$\newcommand{\mdelta}{M_{\Delta}}$
$\newcommand{\rdelta}{R_{\Delta}}$
$\newcommand{\mgs}{M_{\rm gas/star}}$
$\newcommand{\mdeltags}{{\rm M}_{\Delta,{\rm gas/star }} }$
$\newcommand{\mdeltas}{{\rm M}_{\Delta,{\rm star }} }$
$\newcommand{\mdeltag}{{\rm M}_{\Delta,{\rm gas }} }$
$\newcommand{\rhocrit}{\rho_{\rm crit }}$
$\newcommand{\Mfid}{M_{\rm fid}}$
$\newcommand{\sigmamu}{\ensuremath{\sigma_{\mu}}}$
$\newcommand{\varmu}{\ensuremath{\sigma^2_{\mu}}}$
$\newcommand{\varmus}{\ensuremath{\sigma^2_{\mu \vert \mathbf{s} }}}$
$\newcommand{\sigmamus}{\ensuremath{\sigma_{\mu \vert s }}}$
$\newcommand{\sigmamusvec}{\ensuremath{\sigma_{\mu \vert \mathbf{s} }}}$
$\newcommand{\MassScatter}{\sigma_{\Mfivehc \vert \mathbf{s} }}$
$\newcommand{\MassScatterMgas}{\sigma_{\Mfivehc \vert \Mgas}}$
$\newcommand{\MassScatterTsl}{\sigma_{\Mfivehc \vert \Tsl}}$
$\newcommand{\MassScatterTmw}{\sigma_{\Mfivehc \vert \Tmw}}$
$\newcommand{\MassScatterTX}{\sigma_{\Mfivehc \vert \TX}}$
$\newcommand{\MassScatterT}{\sigma_{\Mfivehc \vert T}}$
$\newcommand{\MassScatterYX}{\sigma_{\Mfivehc \vert \YX}}$
$\newcommand{\MassScatterYSZ}{\sigma_{\Mfivehc \vert \YSZ}}$
$\newcommand{\DeltaSS}{\Delta_{{\rm SS}, a}}$
$\newcommand{\kms}{{\rm   km~s}\ensuremath{^{-1}}}$
$\newcommand{\hinv}{\ensuremath{  h^{-1}}}$
$\newcommand{\msol}{\ensuremath{  {\rm M}_\odot}}$
$\newcommand{\msun}{\ensuremath{  {\rm M}_\odot}}$
$\newcommand{\kpc}{\ensuremath{  {\rm kpc}}}$
$\newcommand{\mpc}{\ensuremath{  {\rm Mpc}}}$
$\newcommand{\gpc}{\ensuremath{  {\rm Gpc}}}$
$\newcommand{\gyr}{\ensuremath{  {\rm Gyr}}}$
$\newcommand{\myr}{\ensuremath{  {\rm Myr}}}$
$\newcommand{\hinvmpc}{\ensuremath{  {\rm h, Mpc^{-1}}}}$
$\newcommand{\kev}{\ensuremath{  {\rm keV}}}$
$\newcommand{\Nsat}{\ensuremath{N_{\rm sat}}}$
$\newcommand{\NSat}{\ensuremath{N_{\rm sat}}}$
$\newcommand{\Mstar}{\ensuremath{M_{\rm \star}}}$
$\newcommand{\Mstarsat}{\ensuremath{M_{\rm \star, sat}}}$
$\newcommand{\MstarsatThresh}{\ensuremath{M_{\rm \star, sat}^{\rm th}}}$
$\newcommand{\Mstartot}{\ensuremath{M_{\rm \star, tot}}}$
$\newcommand{\zformation}{\ensuremath{z_{\rm form}}}$
$\newcommand{\sigmaSat}{\ensuremath{\sigma_{\rm sat, 1D}}}$
$\newcommand{\sigmaSatproj}{\ensuremath{\sigma_{\rm sat, 1D}^{\rm proj}}}$
$\newcommand{\sigmaDM}{\ensuremath{\sigma_{\rm DM}}}$
$\newcommand{\vproj}{\ensuremath{v_{\rm sat}^{\rm proj}}}$
$\newcommand{\mlim}{M_{\rm lim}}$
$\newcommand{\lcdm}{\LambdaCDM}$
$\newcommand{\sigmaeight}{\ensuremath{\sigma_8}}$
$\newcommand{\ns}{\ensuremath{n_{\rm s}}}$
$\newcommand{\omegam}{\ensuremath{\Omega_{\rm m}}}$
$\newcommand{\omegab}{\ensuremath{\Omega_{\rm b}}}$
$\newcommand{\omeganow}{\ensuremath{\Omega_{\rm 0}}}$
$\newcommand{\omegaDE}{\ensuremath{\Omega_{\rm DE}}}$
$\newcommand{\omegaLambda}{\ensuremath{\Omega_{\Lambda}}}$
$\newcommand{\Hnow}{\ensuremath{H_{\rm 0}}}$
$\newcommand{\Seight}{\ensuremath{S_8}}$
$\newcommand{\TNGcombined}{\rm TNG300 + TNG-Cluster}$
$\newcommand{\Flam}{\rm FLAMINGO-L1\_m8}$
$\newcommand{\github}[1]{$
$   \href{#1}{\faGithub}$
$}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Mass Proxy Quality of Massive Halo Properties in the IllustrisTNG and FLAMINGO Simulations: I. Hot Gas

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.05176-b31b1b.svg)](https://arxiv.org/abs/2507.05176)<mark>Appeared on: 2025-07-08</mark> -  _23 pages, 14 figures_

</div>
<div id="authors">

E. Aljamal, et al. -- incl., <mark>A. Pillepich</mark>, <mark>J. Braspenning</mark>

</div>
<div id="abstract">

**Abstract:** We examine scale and redshift dependence of mass-property relations (MPRs) for five hot gas properties of two large group- and cluster-scale halo samples realized by the IllustrisTNG, TNG-Cluster and FLAMINGO cosmological hydrodynamical simulations. For intrinsic properties of i) hot gas mass ( $\Mgas$ ), ii) spectroscopic-like temperature ( $\Tsl$ ), iii) soft-band X-ray luminosity ( $\LX$ ), and iv) X-ray ( $\YX$ ) and v) Sunyaev-Zel'dovich ( $\YSZ$ ) thermal energies, we use MPR parameters to infer mass proxy quality (MPQ) --- the implied scatter in total halo mass conditioned on a property --- for halos with $\Mfivehc \geq 10^{13}\msol$ at redshifts, $z \in \{0, 0.5, 1, 2\}$ .We find: (1) in general, scaling relation slopes and covariance display moderate to strong dependence on halo mass, with redshift dependence secondary;(2) for halos with $\Mfivehc > 10^{14}\msol$ , scalings of $\Mgas$ and $\YSZ$ simplify toward self-similar slope and constant intrinsic scatter (5 and 10 \% , respectively) nearly independent of scale, making both measures ideal for cluster finding and characterization to $z=2$ ;(3) halo mass-conditioned likelihoods of hot gas mass and thermal energy at fixed halo mass closely follow a log-normal form;(4) despite normalization differences ranging up to $0.4$ dex, there is good qualitative, and often quantitative, agreement between the scale-dependent slopes and property covariance of the two simulations. Slopes show appreciable redshift dependence at the group scale, while redshift dependence of the scatter is exhibited by low mass FLAMINGO halos only;(5) property correlations are largely consistent between the simulations, with values that mainly agree with existing empirical measurements.We close with a literature survey placing our MPR slopes and intrinsic scatter estimates into context.

</div>

<div id="div_fig1">

<img src="tmp_2507.05176/./plots/MPQ.png" alt="Fig14" width="100%"/>

**Figure 14. -** The mass proxy quality, quantified by the implied halo mass scatter (Equation \eqref{eq:mass-proxy-quality}), for the five gas properties within a 3-D aperture of $\Rfivehc$ listed in the legend
    for TNG (solid lines) and FLAMINGO (dashed) halo populations. Shaded regions are $1\sigma$ uncertainties based on $1000$ bootstrap samples.
    We indicate the low redshift mass regimes of groups and clusters as roughly divided by a halo mass of $10^{14}$\msol$$.  All MPQs show moderate to strong mass dependence.  The relative ordering of mass proxy quality is fairly consistent between the two simulation methods, with thermal energy ($\YSZ$, violet) being the best mass proxy below $10^{14.5} $\msol$$ and hot gas mass ($\Mgas$, black) best above this scale, reaching a minimum of $4\%$ halo mass scatter at $10^{15} $\msol$$. See text for further discussion.
 (*fig:MPQzero*)

</div>
<div id="div_fig2">

<img src="tmp_2507.05176/./plots/slopes.png" alt="Fig15" width="100%"/>

**Figure 15. -** Halo mass and redshift dependence of the MPR slopes for (top to bottom)
    $\Mgas$, $\Tsl$, $\LX$, $\YX$, $\YSZ$.  The
    KLLR-derived values are shown at redshifts $z = 0$(black), $0.5$(blue), $1.0$(gold), and $2$(green) for TNG (left) and FLAMINGO (right) halo populations. Dotted lines indicate the self-similar slopes given in \S\ref{sec:self-similar-theory}. Note that, by definition, the slope in $\YX$ is the sum of the slopes in $\Mgas$ and $\Tsl$.  (*fig:MPR-slope-plots*)

</div>
<div id="div_fig3">

<img src="tmp_2507.05176/./plots/scatters.png" alt="Fig16" width="100%"/>

**Figure 16. -** Halo mass and redshift dependence of the MPR intrinsic scatter values. Format is identical to that of Figure \ref{fig:MPR-slope-plots}. (*fig:MPR-inrinsic-scatter-plots*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.05176"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

136  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
